In [1]:
import requests
import pickle
import time
import networkx as nx
from time import sleep
import matplotlib.pyplot as plt

api_key = "4cf138d1c7a61ceb0ca55c01b032a656"
base_url = "http://ws.audioscrobbler.com/2.0/"

In [2]:
def get_friends(username):

    friends = []
    limit = 50
    page = 1

    while True:
        # call the API and return list of friends
        get_friends_params= {
            'method': 'user.getfriends',
            'user': username,
            'api_key': api_key,
            'format': 'json',
            'limit' : limit,
            'page' : page
        } # read last.fm's documentation about its API at https://www.last.fm/api !

        result=requests.get(base_url, get_friends_params)
        print("sending request to last.fm. page " + str(page))

        if result.status_code == 200: #successful request!
            data = result.json() #thanks to the .json() method of the request library
            page_friends = []
            if 'friends' in data and 'user' in data['friends']:
                page_friends = [curr_user['name'] for curr_user in data['friends']['user']]
            friends.extend(page_friends)
            if len(page_friends) < limit:
                break #no more pages!
            else:
                page += 1 #next page

        else:
            print("Error: code " + str(result.status_code))
            break
            
    return friends


In [3]:
get_friends('ko_tatsu')

sending request to last.fm. page 1
sending request to last.fm. page 2
sending request to last.fm. page 3
sending request to last.fm. page 4
sending request to last.fm. page 5


['amic0gay',
 'cow_puppy',
 'Panopticon91',
 'LuEveLi',
 'FerrariF2001',
 'Deoster',
 'AAAAAAAGHAAAAAH',
 'DavidMetalMusic',
 'computerfire',
 'gant_zgraf',
 'magileine',
 'Cuordileone',
 'Ljlikesbadmusic',
 'sdf5',
 'tmliyfe',
 'vivoinpace',
 'TroensRidder',
 'imightsaysometh',
 'macingtonrs',
 'Miss_Fungina',
 'cloZedspace',
 'ABRS0',
 'Ururururue',
 'eielnoteleien',
 'estreetsycamore',
 'lilloliz',
 'yvngpiss',
 'paryy',
 'Trealeastre',
 'Cairnem',
 'mothercreep_',
 'TheGreep',
 'anisha_',
 'plunkster',
 'Yumekooo',
 'stationinmyroom',
 'evrynoiseatonce',
 'basicbitchtaste',
 'vibrakobra',
 'thesaltysalute',
 'kvet_divizny',
 'Rocketsauce3',
 'mystery0y',
 'Starpentine',
 'vmrrobotic',
 'actuallythesun',
 'will-lane',
 'raskolica',
 'ubernekoneko',
 'Don_Diego',
 'WuK0275',
 'ketchuppacket',
 'zuzu80',
 'coaldingo',
 'colorsandfuel',
 'iljovvv',
 'isisaisat',
 'dream_puppy',
 'claudiogmessina',
 'Leijonanosa',
 'Adeletedperson',
 'snufkinppr',
 'Kkonaice',
 'deathclassicxxr',
 'camu